In [9]:
import os
import time
import json
import praw
from kafka import KafkaProducer

In [2]:

from dotenv import load_dotenv
load_dotenv()

ModuleNotFoundError: No module named 'dotenv'

In [2]:

print(os.getenv("REDDIT_CLIENT_ID"))
print(os.getenv("REDDIT_CLIENT_SECRET"))


ModuleNotFoundError: No module named 'dotenv'

In [10]:
# Environment variables or config
REDDIT_CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")
REDDIT_CLIENT_SECRET = os.getenv('REDDIT_CLIENT_SECRET')
REDDIT_USER_AGENT = "MyRedditApp/0.1/u/tobiOl"
KAFKA_BROKER = os.getenv('KAFKA_BROKER', 'localhost:9092')
TOPIC_NAME = "reddit-tech-posts"

In [11]:
def create_reddit_instance():
    return praw.Reddit(
        client_id=REDDIT_CLIENT_ID,
        client_secret=REDDIT_CLIENT_SECRET,
        user_agent=REDDIT_USER_AGENT
    )

In [12]:
reddit = create_reddit_instance()

MissingRequiredAttributeException: Required configuration setting 'client_id' missing. 
This setting can be provided in a praw.ini file, as a keyword argument to the Reddit class constructor, or as an environment variable.

In [4]:
def main():
    reddit = create_reddit_instance()
    producer = KafkaProducer(
        bootstrap_servers=[KAFKA_BROKER],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    subreddits = ["technology", "programming", "datascience", "MachineLearning"]

    while True:
        for subreddit in subreddits:
            # fetch the top X new submissions
            for submission in reddit.subreddit(subreddit).new(limit=10):
                data = {
                    "id": submission.id,
                    "subreddit": subreddit,
                    "title": submission.title,
                    "selftext": submission.selftext,
                    "created_utc": submission.created_utc
                }
                producer.send(TOPIC_NAME, value=data)

        # Sleep to avoid hitting rate limits and to simulate near real-time
        time.sleep(30)

In [5]:
if __name__ == "__main__":
    main()

MissingRequiredAttributeException: Required configuration setting 'client_id' missing. 
This setting can be provided in a praw.ini file, as a keyword argument to the Reddit class constructor, or as an environment variable.